In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm


model_name = "eryk-mazus/polka-1.1b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}
)


allowed_words = ["skarpetki", "traktor", "ryba", "jabłko", "chleb"]
allowed_ids = [tokenizer.encode(word, add_special_tokens=False) for word in allowed_words]
max_allowed_length = max(len(seq) for seq in allowed_ids)  # Maksymalna długość ciągu w allowed_ids
print("Allowed ids:", allowed_ids)
print("Max allowed sequence length:", max_allowed_length)


few_shot_examples = (
    "Zagadka: Co szczeka i jest najlepszym przyjacielem człowieka? Odpowiedź to jedno słowo: pies\n"
    "Zagadka: Co ma cztery nogi i jest zwierzęciem domowym? Odpowiedź to jedno słowo: kot\n"
    "Zagadka: Co jest przezroczyste, cieczy i jest niezbędne do życia? Odpowiedź to jedno słowo: woda\n\n"
)


riddles = [
    "Zagadka: Co jest słodkie, czerwone i można je zjeść? Odpowiedź to jedno słowo:",
    "Zagadka: Co to za część garderoby, którą zakładamy na stopy przed założeniem butów? Odpowiedź to jedno słowo:",
    "Zagadka: Co to za pokarm, powstaje z mąki i wody? Odpowiedź to jedno słowo:"
]


prompts = [few_shot_examples + riddle for riddle in riddles]

def generate_single_word(model, input_ids, allowed_ids, max_length):
    output_ids = input_ids.clone()
    valid_word = False


    allowed_tokens = list(set([token for seq in allowed_ids for token in seq]))

    while not valid_word and output_ids.shape[1] - input_ids.shape[1] <= max_length:
        with torch.no_grad():
            logits = model(output_ids).logits[:, -1, :] #(batch_size=1 (bo generujemy jenda odpowiedz), sequence_length, vocab_size)
            # maskujemy wszystkie tokeny, które nie należą do allowed_tokens
            mask = torch.full_like(logits, float("-inf"))
            mask[:, allowed_tokens] = logits[:, allowed_tokens]  # pozostawiamy tylko dozwolone tokeny
            logits = mask
            # wybieramy token o najwyższym prawdopodobieństwie spośród allowed_tokens
            top_token = torch.argmax(logits, dim=-1, keepdim=True)
            output_ids = torch.cat([output_ids, top_token], dim=-1)
            print("Generated token:", top_token)

        # sprawdzanie, czy końcowa sekwencja odpowiada któremuś z allowed_ids
        for allowed_seq in allowed_ids:
            if output_ids[0, -len(allowed_seq):].tolist() == allowed_seq:
                valid_word = True
                return tokenizer.decode(allowed_seq)  # zwracamy pełne słowo jako tekst


all_responses = {}

for prompt, riddle in zip(prompts, riddles):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(model.device)

    responses = []
    for _ in tqdm(range(3), desc=f"Processing riddle: '{riddle[:50]}...'"):
        response = generate_single_word(model, input_ids, allowed_ids, max_allowed_length)
        responses.append(response)

    all_responses[riddle] = responses


for riddle, responses in all_responses.items():
    print(f"{riddle}\nWygenerowane odpowiedzi: {responses}\n")


Allowed ids: [[2071, 6834, 300, 1984], [1020, 10407], [24721, 2291], [432, 370, 30006, 2901], [521, 19982]]
Max allowed sequence length: 4


Processing riddle: 'Zagadka: Co jest słodkie, czerwone i można je zjeś...':   0%|          | 0/3 [00:00<?, ?it/s]

Generated token: tensor([[432]])
Generated token: tensor([[370]])
Generated token: tensor([[30006]])


Processing riddle: 'Zagadka: Co jest słodkie, czerwone i można je zjeś...':  33%|███▎      | 1/3 [00:28<00:57, 28.87s/it]

Generated token: tensor([[2901]])
Generated token: tensor([[432]])
Generated token: tensor([[370]])
Generated token: tensor([[30006]])


...

Generated token: tensor([[19982]])
Generated token: tensor([[521]])


Processing riddle: 'Zagadka: Co to za pokarm, powstaje z mąki i wody? ...':  67%|██████▋   | 2/3 [00:28<00:14, 14.09s/it]

Generated token: tensor([[19982]])
Generated token: tensor([[521]])


Processing riddle: 'Zagadka: Co to za pokarm, powstaje z mąki i wody? ...': 100%|██████████| 3/3 [00:40<00:00, 13.63s/it]

Generated token: tensor([[19982]])
Zagadka: Co jest słodkie, czerwone i można je zjeść? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['jabłko', 'jabłko', 'jabłko']

Zagadka: Co to za część garderoby, którą zakładamy na stopy przed założeniem butów? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['skarpetki', 'skarpetki', 'skarpetki']

Zagadka: Co to za pokarm, powstaje z mąki i wody? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['chleb', 'chleb', 'chleb']



In [ ]:

model_name = 'flax-community/papuGaPT2'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}
)


allowed_words = ["skarpetki", "traktor", "ryba", "jabłko", "chleb"]
allowed_ids = [tokenizer.encode(word, add_special_tokens=False) for word in allowed_words]
max_allowed_length = max(len(seq) for seq in allowed_ids)  # Maksymalna długość ciągu w allowed_ids
print("Allowed ids:", allowed_ids)
print("Max allowed sequence length:", max_allowed_length)


all_responses = {}

for prompt, riddle in zip(prompts, riddles):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(model.device)

    responses = []
    for _ in tqdm(range(3), desc=f"Processing riddle: '{riddle[:50]}...'"):
        response = generate_single_word(model, input_ids, allowed_ids, max_allowed_length)
        responses.append(response)

    all_responses[riddle] = responses


for riddle, responses in all_responses.items():
    print(f"{riddle}\nWygenerowane odpowiedzi: {responses}\n")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Allowed ids: [[5702, 483, 946], [7630, 716], [335, 491], [317, 70, 9735], [274, 7012]]
Max allowed sequence length: 3


Processing riddle: 'Zagadka: Co jest słodkie, czerwone i można je zjeś...':   0%|          | 0/3 [00:00<?, ?it/s]

Generated token: tensor([[317]])
Generated token: tensor([[70]])


Processing riddle: 'Zagadka: Co jest słodkie, czerwone i można je zjeś...':  33%|███▎      | 1/3 [00:02<00:04,  2.10s/it]

...

Generated token: tensor([[9735]])
Generated token: tensor([[317]])
Generated token: tensor([[70]])


Processing riddle: 'Zagadka: Co to za pokarm, powstaje z mąki i wody? ...':  67%|██████▋   | 2/3 [00:03<00:01,  1.54s/it]

Generated token: tensor([[9735]])
Generated token: tensor([[317]])
Generated token: tensor([[70]])


Processing riddle: 'Zagadka: Co to za pokarm, powstaje z mąki i wody? ...': 100%|██████████| 3/3 [00:05<00:00,  1.73s/it]

Generated token: tensor([[9735]])
Zagadka: Co jest słodkie, czerwone i można je zjeść? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['jabłko', 'jabłko', 'jabłko']

Zagadka: Co to za część garderoby, którą zakładamy na stopy przed założeniem butów? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['jabłko', 'jabłko', 'jabłko']

Zagadka: Co to za pokarm, powstaje z mąki i wody? Odpowiedź to jedno słowo:
Wygenerowane odpowiedzi: ['jabłko', 'jabłko', 'jabłko']

